In [2]:
import casadi as ca
import numpy as np
from mpc_model import Model
from mpc_simulation import  Simulator
from mpc_solver import MPC
from mpc_plot import Plot
import matplotlib.pyplot as plt

In [3]:
def furuta_ode(t, x, u, p, z):
    """
    Furuta Pendulum
    """
    # Parameter configuration
    alpha = p[0]
    beta =  p[1]
    delta = p[2]
    gamma = p[3]

    dx1_dt = x[1]
    dx2_dt = 1 / (alpha * beta - gamma ** 2 + (beta ** 2 + gamma ** 2) * ca.sin(x[2]) ** 2) * (
                beta * gamma * (ca.sin(x[2] ** 2 - 1)) * ca.sin(x[2]) * x[1] ** 2 - 2 * beta ** 2 * ca.cos(
            x[2]) * ca.sin(x[2]) * x[1] * x[3] + beta * gamma * ca.sin(x[2]) * x[3] ** 2 - gamma * delta * ca.cos(
            x[2]) * ca.sin(x[2]) + beta * u[0] - gamma * ca.cos(x[2]) * u[1])
    dx3_dt = x[3]
    dx4_dt = 1 / (alpha * beta - gamma ** 2 + (beta ** 2 + gamma ** 2) * ca.sin(x[2]) ** 2) * (
                beta * (alpha + beta * ca.sin(x[2]) ** 2) * ca.cos(x[2]) * ca.sin(x[2]) * x[
            1] ** 2 + 2 * beta * gamma * (1 - ca.sin(x[2]) ** 2) * ca.sin(x[2]) * x[1] * x[3] - gamma ** 2 * ca.cos(
            x[2]) * ca.sin(x[2]) * x[3] ** 2 + delta * (alpha + beta * ca.sin(x[2]) ** 2) * ca.sin(
            x[2]) - gamma * ca.cos(x[2]) * u[0] + (alpha + beta * ca.sin(x[2]) ** 2) * u[1])

    rhs = [dx1_dt,
           dx2_dt,
           dx3_dt,
           dx4_dt
           ]
    return ca.vertcat(*rhs)

In [4]:
def furuta_measure(t, x, z):
    """
    Furuta Pendulum measurement
    """
    
    rhs = [x[0],
           x[2],
           x[2],
           x[3]
           ]
    return ca.vertcat(*rhs)

In [5]:
def set_constraint(N_pred):
    # set constraint
    lbx = [0, 0, -ca.pi, 0]
    ubx = [0, 0, -ca.pi, 0]
    lbx += [-ca.pi, -ca.inf, -ca.pi, -ca.inf, -ca.inf, -ca.inf] * N_pred
    ubx += [ca.pi, ca.inf, 2 * ca.pi, ca.inf, ca.inf, ca.inf] * N_pred

    x0 = [0, 0, -ca.pi, 0]
    x0 += [0, 0, 0, 0, 0, 0] * N_pred

    lbg = [0, 0, 0, 0] * N_pred
    ubg = [0, 0, 0, 0] * N_pred

    p = [0, 0, 0, 0, 0, 0, 0] # t_var, xr_var, ur_var

    return lbx, ubx, lbg, ubg, p, x0

In [31]:
a = ca.SX.sym('a',2,2)
I = ca.SX.eye(2)
ca.solve(a,I)

SX(@1=((a_0*a_3)-(a_2*a_1)), 
[[(a_3/@1), (-(a_2/@1))], 
 [(-(a_1/@1)), (a_0/@1)]])

In [33]:
a = ca.SX.sym('a',2,2)
sqrt = ca.chol(a)
sqrt  @ sqrt.T

SX(@1=(a_2/a_0), @2=(sqrt(a_0)*@1), @3=(a_3-(a_2*@1)), @4=sqrt(@3), 
[[(a_0+sq(@2)), (@2*@4)], 
 [(@4*@2), @3]])

In [ ]:
class stochastic
    def __init__(self, model, measurement = None, opt = None):
        Nx = model.Nx
        Nu = model.Nu
        self.Nx = Nx
        self.Nu = Nu
        
        t = model.t
        x = model.x
        u = model.u
        z = model.z 
        p = model.p         
        
        y = measurement(t, x, z)
        self.Ny = y.shape[0]
        self.y = y
        v = ca.SX.sym("v",Ny)    #  Measurement noise
        w = ca.SX.sym("w",Nx)    #  Process noise
        
        measure_nom_func = ca.Function("measure_nom_func", [t,x,z], [y])
        measure_noised_func = ca.Function("measure_noised_func", [t, x, z, v], [y + v])    #  Noise measurement
        self.measure_nom_func = measure_nom_func
        self.measure_noised_func = measure_noised_func
        
        ode_dis_func = model.ode_dis_func
        self.ode_dis_func = ode_dis_func
        self.ode_noised_func = ca.Function("ode_noised_func", [t, x, u, p, z, w], [ode_dis_func(t, x, u, p, z) + w])    #  Noise measurement 
        
        
        if opt == None:
            opt = {}
            
            
        if 'measure_noise' in opt:

        if 'process_noise' in opt:

        if 'method' in opt:
            if opt['method'] == 'UKF':
                self.init_ukf()
                self.propogation()
                
                
                
    def init_ukf(self,alpha = 0.4,beta = 2,kappa = 0.1):
        Nx = self.Nx
        Ny = self.Ny
        ode_dis_func = self. ode_dis_func
        measure_nom_func = self.measure_nom_func
        
        L = 2 * Nx +1
        lambda_ = alpha**2 * (L + kappa) - L
        omega_m_0 = 1 / (L + lambda_)
        omega_c_0 = 1 / (L + lambda_) + (1 - alpha**2 + beta)
        omega_m_i = 1 / (2 * (L + lambda_))
        omega_c_i = 1 / (2 * (L + lambda_))
        
        
        x = ca.SX.sym("x",Nx)
        x_var_sqrt = ca.SX.sym("x_var_sqrt",Nx, Nx)
        y_var_inv = ca.SX.sym("y_var_inv", Ny, Ny)
        y_measure = ca.SX.sym("y_measure", Ny)
        w_var = ca.SX.sym("w_var",Nx, Nx)
        v_var = ca.SX.sym("v_var",Ny, Ny)
        
        
        x_matrix_temp = ca.SX.zeros(Nx, Nx)
        for i in range(Nx):
            x_matrix_temp[:,i] = x
        sigma_point_matrix = ca.horzcat(x, x_matrix_temp + ca.sqrt(L + lambda_) * x_var_sqrt, x_matrix_temp - ca.sqrt(L + lambda_) * x_var_sqrt)
        sigma_point_matrix_func = ca.Function('sigma_point_matrix_func', [x, x_var_sqrt],[sigma_point_matrix])
        self.sigma_point_matrix_func = sigma_point_matrix_func
        
        
        # Update mean state
        sigma_point_prop = ode_dis_func(t, sigma_point_matrix[:,0], u, p, z)
        x_mean = omega_m_0 * sigma_point_prop
        for i in range(2 * L):
            sigma_point_prop = ode_dis_func(t, sigma_point_matrix[:,i + 1], u, p, z)
            x_mean += omega_m_i * sigma_point_prop
        x_mean_func = ca.Function('x_mean_func',[t, x, x_var_sqrt, u, p, z],[x_mean])
        # Update state covariance
        sigma_point_prop = ode_dis_func(t, sigma_point_matrix[:,0], u, p, z)
        x_var = omega_c_0 * (sigma_point_prop - x_mean) @ (sigma_point_prop - x_mean).T
        for i in range(2 * L):
            sigma_point_prop = ode_dis_func(t, sigma_point_matrix[:,i + 1], u, p, z)
            x_var += omega_c_i * (sigma_point_prop - x_mean) @ (sigma_point_prop - x_mean).T
        x_var += w_var
        x_var_func = ca.Function('x_var_func',[t, x, x_var_sqrt, u, p, z, w_var], [x_var])
        # Update observation mean
        phi_prop = measure_nom_func(t, sigma_point_matrix[:,0], z)
        y_mean = omega_m_0 * phi_prop
        for i in range(2 * L):
            phi_prop = measure_nom_func(t, sigma_point_matrix[:,i + 1], z)
            y_mean += omega_m_i * phi_prop
        y_mean_func = ca.Function('y_mean_func',[t, x, x_var_sqrt, z],[y_mean])
        # Update covariance_y and covariance_xy
        sigma_point_prop = ode_dis_func(t, sigma_point_matrix[:,0], u, p, z)
        phi_prop = measure_nom_func(t, sigma_point_matrix[:,0], z)        
        y_var = omega_c_0 * (phi_prop - y_mean) @ (phi_prop - y_mean).T
        xy_var = omega_c_0 * (sigma_point_prop - x_mean) @ (phi_prop - y_mean).T
        for i in range(2 * L):
            sigma_point_prop = ode_dis_func(t, sigma_point_matrix[:,i+1], u, p, z)
            phi_prop = measure_nom_func(t, sigma_point_matrix[:,i+1], z)
            y_var = omega_c_i * (phi_prop - y_mean) @ (phi_prop - y_mean).T
            xy_var = omega_c_i * (sigma_point_prop - x_mean) @ (phi_prop - y_mean).T
        y_var += v_var
        y_var_func = ca.Function('y_var_func',[t, x, x_var_sqrt, z, v_var], [y_var])
        xy_var_func = ca.Function('xy_var_func',[t, x, x_var_sqrt, u, p, z], [xy_var]) 
        # Update state from available measurements
        K = xy_var @ y_var_inv
        x_post = x_mean + K @ (y - y)

In [ ]:
Nt = 1
Nx = 4
Nu = 2
Np = 0
Nz = 0

delta_t = 0.01
N_pred = 50
N_sim = 100

t_SX = ca.SX.sym("t_SX", Nt)
x_SX = ca.SX.sym("x_SX", Nx)
u_SX = ca.SX.sym("u_SX", Nu)
p_SX = ca.SX.sym("p_SX", Np)

z_SX = ca.SX.sym("z_SX", Nz)


alpha = 0.0033472
beta =  0.0038852
delta = 0.097625
gamma = 0.0024879

para = [alpha,beta,delta,gamma]

furuta_ode(t_SX, x_SX, u_SX, para, z_SX)
lbx, ubx, lbg, ubg, p, x0 = set_constraint(N_pred)


xr_SX = ca.SX.sym("xr_SX", Nx)
ur_SX = ca.SX.sym("ur_SX", Nu)
Q = np.diag([10, 1, 100, 1])
R = np.diag([1, 1])
Q_f = np.diag([0, 0, 0, 0])
stage_cost = (x_SX - xr_SX).T @ Q @ (x_SX - xr_SX) + (u_SX - ur_SX).T @ R @ (u_SX - ur_SX)    #  Lagrange term
terminal_cost = (x_SX - xr_SX).T @ Q_f @ (x_SX - xr_SX)    #  Mayer term
stage_cost_func = ca.Function("stage_cost_func",[x_SX, xr_SX, u_SX, ur_SX, z_SX], [stage_cost])
terminal_cost_func = ca.Function("terminal_cost_func",[x_SX, xr_SX], [terminal_cost])

model = Model(t_SX, x_SX, u_SX, z_SX, p_SX, delta_t, para=para, ode=furuta_ode, alg=None, opt=None,
              stage_cost_func=stage_cost_func, terminal_cost_func=terminal_cost_func)

solver_opt = {}
solver_opt['print_time'] = False
solver_opt['ipopt'] = {
    'max_iter': 500,
    'print_level': 1,
    'acceptable_tol': 1e-6,
    'acceptable_obj_change_tol': 1e-6
}

opt = {}
opt['solver_opt'] = solver_opt

mpc_solver = MPC(model, N_pred, opt = opt)
mpc_simulator = Simulator(model, mpc_solver, N_sim, lbx, ubx, lbg, ubg, p, x0)